## Symbolic Mathematics

In [1]:
import numpy as np

We now introduce symbolic computation, which can be applied beneficially in many areas of finance. Let us import <code>SymPy</code>, the library specifically dedicated to symbolic computation:

In [2]:
import sympy as sy

_Symbolic Mathematics_ >
### Basics

<code>SymPy</code> introduces new classes of objects. A fundamental class is the <code>Symbol</code> class:

In [3]:
x = sy.Symbol('x')
y = sy.Symbol('y')

In [4]:
type(x)

sympy.core.symbol.Symbol

Like <code>NumPy</code>, <code>SymPy</code> has a number of (mathematical) function definitions.

In [5]:
sy.sqrt(x)

sqrt(x)

Although <code>x</code> has no numerical value, the square root of <code>x</code> is nevertheless defined with <code>SymPy</code> since <code>x</code> is a <code>Symbol</code> object. <code>sy.sqrt(x)</code> can be part of arbitrary mathematical expressions. Notice that <code>SymPy</code> in general automatically simplifies a given mathematical expression:

In [6]:
3 + sy.sqrt(x) - 4 ** 2

sqrt(x) - 13

In [7]:
f = x ** 2 + 3 + 0.5 * x ** 2 + 3 / 2

In [8]:
sy.simplify(f)

1.5*x**2 + 4.5

<code>SymPy</code> provides three basic renderers for mathematical expressions:
- <code>LaTex</code>-based
- <code>Unicode</code>-based
- <code>ASCII</code>-based

When working solely in the <code>IPython Notebook</code>, <code>LaTeX</code> rendering is generally a good (i.e., visually appealing) choice. We stick to the simplest option, <code>ASCII</code>, to illustrate that there is no handmade type setting involved:

In [9]:
sy.init_printing(pretty_print=False, use_unicode=False)

In [10]:
 print(sy.pretty(f))

     2      
1.5*x  + 4.5


In [11]:
print(sy.pretty(sy.sqrt(x) + 0.5))

  ___      
\/ x  + 0.5


SymPy also provides many other useful mathematical functions — for example, when it comes to numerically evaluating. 

In [12]:
pi_str = str(sy.N(sy.pi, 400000))
pi_str[:40]

'3.14159265358979323846264338327950288419'

In [13]:
pi_str[-40:]

'8245672736856312185020980470362464176198'

You can also look up your birthday if you wish.

In [14]:
pi_str.find('540903')

310409

_Symbolic Mathematics_ >
### Equations

A strength of <code>SymPy</code> is solving equations, e.g., of the form $x^{2} – 1 = 0$:

In [15]:
sy.solve(x ** 2 - 1)

[-1, 1]

<code>SymPy<?code> presumes that you are looking for a solution to the equation obtained by
equating the given expression to zero. Therefore, equations like $x^{2} – 1 = 3$ might have to be reformulated to get the desired result:

In [16]:
sy.solve(x ** 2 - 1 - 3)

[-2, 2]

<code>SymPy</code> can cope with more complex expressions, like $x^{3} + 0.5x^{2} – 1 = 0$:

In [17]:
sy.solve(x ** 3 + 0.5 * x ** 2 - 1)

[0.858094329496553, -0.679047164748276 - 0.839206763026694*I, -0.679047164748276 + 0.839206763026694*I]

There is obviously no guarantee of a solution, either from a mathematical point of view (i.e., the existence of a solution) or from an algorithmic point of view (i.e., an implementation).

<code>SymPy</code> works similarly with functions exhibiting more than one input parameter, and to this end also with complex numbers. As a simple example take the equation $x^{2} + y^{2} = 0$:

In [18]:
sy.solve(x ** 2 + y ** 2)

[{x: -I*y}, {x: I*y}]

_Symbolic Mathematics_ >
### Integration

Another strength of <code>SymPy</code> is integration and differentiation. We revisit the example function used for numerical- and simulation-based integration and derive now both a symbolic and a numerically exact solution. We need symbols for the integration limits:

In [19]:
 a, b = sy.symbols('a b')

Having defined the new symbols, we can “pretty print” the symbolic integral:

In [20]:
print(sy.pretty(sy.Integral(sy.sin(x) + 0.5 * x, (x, a, b))))

  b                    
  /                    
 |                     
 |  (0.5*x + sin(x)) dx
 |                     
/                      
a                      


Using <code>integrate</code>, we can then derive the _antiderivative_ of the integration function:

In [21]:
int_func = sy.integrate(sy.sin(x) + 0.5 * x, x)

In [22]:
print(sy.pretty(int_func))

      2         
0.25*x  - cos(x)


Equipped with the antiderivative, the numerical evaluation of the integral is only three steps away. To numerically evaluate a <code>SymPy</code> expression, replace the respective symbol with the numerical value using the method <code>subs</code> and call the method <code>evalf</code> on the new expression:

In [23]:
Fb = int_func.subs(x, 9.5).evalf()
Fa = int_func.subs(x, 0.5).evalf()

The difference between <code>Fb</code> and <code>Fa</code> then yields the exact integral value:

In [24]:
Fb - Fa # exact value of integral

24.3747547180867

The integral can also be solved symbolically with the symbolic integration limits:

In [25]:
int_func_limits = sy.integrate(sy.sin(x) + 0.5 * x, (x, a, b))
print(sy.pretty(int_func_limits))

        2         2                  
- 0.25*a  + 0.25*b  + cos(a) - cos(b)


Using a <code>dict</code> object for multiple substitutions — and evaluation then yields the integral value:

In [26]:
int_func_limits.subs({a : 0.5, b : 9.5}).evalf()

24.3747547180868

Providing quantified integration limits yields the exact value in a single step:

In [27]:
sy.integrate(sy.sin(x) + 0.5 * x, (x, 0.5, 9.5))

24.3747547180867

_Symbolic Mathematics_ >
### Differentiation

The derivative of the antiderivative shall yield in general the original function. Let us check this by applying the <code>diff</code> function to the symbolic antiderivative from before:

In [28]:
int_func.diff()

0.5*x + sin(x)

We want to use differentiation now to derive the exact solution of the convex minimization problem. To this end, we define the respective function symbolically as follows:

In [29]:
f = (sy.sin(x) + 0.05 * x ** 2 
     + sy.sin(y) + 0.05 * y ** 2)

For the minimization, we need the two partial derivatives with respect to both variables, <code>x</code> and <code>y</code>:

In [30]:
del_x = sy.diff(f, x)
del_x

0.1*x + cos(x)

In [31]:
del_y = sy.diff(f, y)
del_y

0.1*y + cos(y)

A necessary but not sufficient condition for a global minimum is that both partial derivatives are zero. There is no guarantee of a symbolic solution. Both algorithmic and (multiple) existence issues come into play here. However, we can solve the two equations numerically, providing “educated” guesses based on the global and local minimization efforts from before:

In [32]:
xo = sy.nsolve(del_x, -1.5)
xo

mpf('-1.4275517787645941')

In [33]:
yo = sy.nsolve(del_y, -1.5)
yo

mpf('-1.4275517787645941')

In [34]:
f.subs({x : xo, y : yo}).evalf()

-1.77572565314742

Providing uneducated/arbitrary guesses might trap the algorithm in a local minimum instead of the global one:

In [35]:
xo = sy.nsolve(del_x, 1.5)
xo

mpf('1.7463292822528528')

In [36]:
yo = sy.nsolve(del_y, 1.5)
yo

mpf('1.7463292822528528')

In [37]:
f.subs({x : xo, y : yo}).evalf()
  # local minimum

2.27423381055640

> <center>**SYMBOLIC COMPUTATIONS**</center>

> When doing mathematics with Python, you should always think of SymPy and symbolic computations. Especially for interactive financial analytics, this can be a more efficient approach compared to non-symbolic approaches.